In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize, Compose
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import utils



In [27]:

transform = Compose([
    Resize((128, 256)),
    ToTensor()
])


target_transform = Compose([
    Resize((128, 256)),
    ToTensor()
])

#Desktop
dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#Laptop
#dataset = Cityscapes(root="C:/Users/20182573/Documents/CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#subset_small, subset_big = random_split(dataset, [0.5,0.5])
train_dataset, val_dataset = random_split(dataset, [0.8,0.2])

In [28]:
len(train_dataset)

2380

In [29]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size) #num_workers=8
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [6]:

from model import Model

model = Model().cuda()
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (IMG,SEGM) in enumerate(dataloader):
        IMG = IMG.to('cuda')
        SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
        SEGM = utils.map_id_to_train_id(SEGM).to('cuda')
        
        #predict
        pred = model(IMG)
        #Loss
        loss = loss_fn(pred, SEGM)

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(IMG)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [8]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (IMG,SEGM) in enumerate(dataloader):
            IMG = IMG.to('cuda')
            SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
            SEGM = utils.map_id_to_train_id(SEGM).to('cuda')

            pred = model(IMG)
            test_loss += loss_fn(pred, SEGM).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")


In [30]:
epochs = 10 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.282718  [   64/ 2380]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "model.pth")